In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from DrawtexDataset import DrawtexDataset

#### Hyperparameters

In [2]:
TRAIN_BATCH_SZ = 128
TEST_BATCH_SZ = 1000
EPOCHS = 1
LEARN_RATE = 0.01
CLASS_CNT = 78

device: str = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


#### Model definition

In [3]:
class DrawtexModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(1, 150, 9, bias=False)  # 45x45 -> 37x37
        self.conv1_bn = nn.BatchNorm2d(150)
        self.conv2 = nn.Conv2d(150, 200, 9, bias=False)  # 37x37 -> 29x29
        self.conv2_bn = nn.BatchNorm2d(200)
        self.conv3 = nn.Conv2d(200, 300, 9, bias=False)  # 29x29 -> 21x21
        self.conv3_bn = nn.BatchNorm2d(300)
        self.conv4 = nn.Conv2d(300, 500, 9, bias=False)  # 21x21 -> 13x13
        self.conv4_bn = nn.BatchNorm2d(500)
        self.conv5 = nn.Conv2d(500, 800, 9, bias=False)  # 13x13 -> 5x5
        self.conv5_bn = nn.BatchNorm2d(800)
        self.lin1 = nn.Linear(20000, 78, bias=False)
        self.lin1_bn = nn.BatchNorm1d(78)

    def forward(self, x: torch.Tensor):
        x: torch.Tensor = self.relu(self.conv1_bn(self.conv1(x)))
        x: torch.Tensor = self.relu(self.conv2_bn(self.conv2(x)))
        x: torch.Tensor = self.relu(self.conv3_bn(self.conv3(x)))
        x: torch.Tensor = self.relu(self.conv4_bn(self.conv4(x)))
        x: torch.Tensor = self.relu(self.conv5_bn(self.conv5(x)))
        x = torch.flatten(x.permute(0, 2, 3, 1), 1)
        x = self.lin1_bn(self.lin1(x))
        return x

#### Dataloader setup

In [4]:
data_set = DrawtexDataset(transforms.ToTensor())
TRAIN_SIZE = int(0.8 * len(data_set))
TEST_SIZE = len(data_set) - TRAIN_SIZE
train_set, test_set = torch.utils.data.random_split(data_set, [TRAIN_SIZE, TEST_SIZE])

train_loader = DataLoader(
    dataset=train_set,
    batch_size=TRAIN_BATCH_SZ,
    shuffle=True,
    num_workers=4
)

test_loader = DataLoader(
    dataset=test_set,
    batch_size=TEST_BATCH_SZ,
    shuffle=False,
    num_workers=4
)

model = DrawtexModel().to(device)
print(model)

DrawtexModel(
  (relu): ReLU()
  (conv1): Conv2d(1, 150, kernel_size=(9, 9), stride=(1, 1), bias=False)
  (conv1_bn): BatchNorm2d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(150, 200, kernel_size=(9, 9), stride=(1, 1), bias=False)
  (conv2_bn): BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(200, 300, kernel_size=(9, 9), stride=(1, 1), bias=False)
  (conv3_bn): BatchNorm2d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(300, 500, kernel_size=(9, 9), stride=(1, 1), bias=False)
  (conv4_bn): BatchNorm2d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(500, 800, kernel_size=(9, 9), stride=(1, 1), bias=False)
  (conv5_bn): BatchNorm2d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lin1): Linear(in_features=20000, out_features=326, bias=False)
  (lin1_bn): BatchNorm1d(326, eps=1e-05, momentum=0.1, 

In [5]:
tens = torch.rand((2, 1, 45, 45)).to(device)

output1 = model(tens)

print(output1)

tensor([[ 0.9885,  0.9768, -0.9998,  0.9991,  1.0000,  1.0000,  0.9999,  1.0000,
         -0.9887, -0.9999, -0.9997, -0.9999, -0.9999, -0.9999, -0.9996, -0.9997,
          0.9999, -0.9999, -0.9999,  0.9999, -1.0000,  0.9999, -0.9939,  0.9945,
         -0.9997,  0.9994, -0.9993,  0.9998,  0.9932, -1.0000,  0.9995, -0.9998,
          1.0000, -0.9998, -0.9995, -0.9992,  0.9994,  0.9929,  0.9999,  0.9966,
         -0.9987,  0.9994,  1.0000, -0.9078,  0.9990,  0.9954,  0.9998, -0.9999,
         -0.9998, -1.0000, -0.9998, -0.9960,  0.9993,  0.9998, -0.9998, -0.9980,
          0.9999,  0.9999, -1.0000, -0.9999, -0.9998,  0.9984, -0.9999,  0.9986,
         -1.0000,  0.9989,  0.9977,  1.0000,  0.9329,  0.9987,  0.9988, -0.9998,
         -0.9815,  0.9999, -0.9993, -0.9996,  0.9986,  0.9999, -0.9989, -0.9988,
         -0.9999,  0.9999, -0.9999, -0.9947, -0.9999,  0.9968,  1.0000, -0.9999,
         -0.9999,  0.9997,  0.9638,  0.9983,  0.9901,  0.9963,  0.9994,  1.0000,
         -0.9908, -0.9997, -

#### Training

In [6]:
def train():
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=LEARN_RATE)
    steps = len(train_loader)
    for epoch in range(EPOCHS):
        for i, (img, label) in enumerate(train_loader):
            img: torch.Tensor = img.to(device, non_blocking=True)
            label: torch.Tensor = label.to(device, non_blocking=True)
            output = model(img).to(device)
            loss: torch.Tensor = criterion(output, label).to(device)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % 10 == 0:
                print(f"Epoch {epoch + 1}/{EPOCHS}, Batch {i + 1}/{steps}, Loss {loss.item():.4f}")

    torch.save(model.state_dict(), "./DrawtexModel.pth")
    torch.save(optimizer.state_dict(), "./Optimizer.pth")

train()


Epoch 1/1, Batch 1/1208, Loss 6.2406


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\benjx\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\benjx\AppData\Local\Temp/ipykernel_16504/3428228553.py", line 22, in <module>
    train()
  File "C:\Users\benjx\AppData\Local\Temp/ipykernel_16504/3428228553.py", line 13, in train
    loss.backward()
  File "c:\program files\python39\lib\site-packages\torch\_tensor.py", line 363, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "c:\program files\python39\lib\site-packages\torch\autograd\__init__.py", line 173, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\benjx\AppData\Roaming\Python\Python39\site-

TypeError: object of type 'NoneType' has no len()

#### Testing

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for img, labels in test_loader:
        img = img.to(device, non_blocking= True)
        labels = labels.to(device, non_blocking= True)

        output = model(img)
        _, prediction = torch.max(output, 1)
        total += labels.size(0)
        correct += (prediction == labels).sum().item()

    acc = 100.0 * correct / total
    print(f"Accuracy: {acc}%")
